In [110]:
!pip install tensorflow opencv-python-headless numpy scikit-learn

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
import os
import glob
import time

# Define the Drive location where the dataset is already stored
extracted_dir = '/content/drive/My Drive/fyp dataset egogesture'

# Verify the directory exists
if os.path.exists(extracted_dir):
    print(f"Extracted directory found: {extracted_dir}")
    # Force refresh of directory listing
    time.sleep(2)  # Delay to ensure sync
    print(f"Contents of {extracted_dir}: {os.listdir(extracted_dir)}")

    # Verify videos folder
    videos_dir = os.path.join(extracted_dir, 'videos')
    if os.path.exists(videos_dir):
        video_files = glob.glob(os.path.join(videos_dir, '*'))
        print(f"Videos folder contains {len(video_files)} items (subfolders/files):")
        for file in video_files[:5]:  # Show first 5 for brevity
            print(f"  - {os.path.basename(file)}")
        if len(video_files) > 5:
            print(f"  - ... and {len(video_files) - 5} more items")
    else:
        print("Videos folder not found.")

    # Verify labels folder with recursive search
    labels_dir = os.path.join(extracted_dir, 'labels')
    print(f"Checking labels directory: {labels_dir}")
    if os.path.exists(labels_dir):
        print(f"Labels directory exists. Contents: {os.listdir(labels_dir)}")
        # Check all files in labels and subdirectories
        all_label_files = glob.glob(os.path.join(labels_dir, '**'), recursive=True)
        print(f"All files/folders in labels (including subdirectories): {len(all_label_files)}")
        for file in all_label_files[:5]:  # Show first 5 for brevity
            print(f"  - {os.path.basename(file)} (full path: {file})")
        if len(all_label_files) > 5:
            print(f"  - ... and {len(all_label_files) - 5} more items")
        # Check for .csv files recursively
        label_csv_files = glob.glob(os.path.join(labels_dir, '**', '*.csv'), recursive=True)
        print(f"Total .csv files in labels: {len(label_csv_files)}")
        for file in label_csv_files[:5]:  # Show first 5 for brevity
            print(f"  - {os.path.basename(file)} (full path: {file})")
        if len(label_csv_files) > 5:
            print(f"  - ... and {len(label_csv_files) - 5} more files")
        if len(label_csv_files) != 8:  # Adjust based on expected number
            print(f"Warning: Expected 8 .csv files based on screenshot, but found {len(label_csv_files)}.")
    else:
        print("Labels folder not found. Directory does not exist.")
else:
    print("Extracted directory not found. Ensure the 'fyp dataset egogesture' folder is correctly set up in Google Drive and Drive is mounted.")

Extracted directory found: /content/drive/My Drive/fyp dataset egogesture
Contents of /content/drive/My Drive/fyp dataset egogesture: ['labels', 'videos', 'all_features.npy', 'gesture_recognition_model.h5', 'label_map.npy', 'unique_labels.npy']
Videos folder contains 5 items (subfolders/files):
  - videos_1-001
  - videos_2-002
  - videos_3-004
  - videos_4-003
  - videos_5-003
Checking labels directory: /content/drive/My Drive/fyp dataset egogesture/labels
Labels directory exists. Contents: ['subject08', 'subject13', 'subject09', 'subject06', 'subject14', 'subject15', 'subject12', 'subject10', 'subject11', 'subject07', 'subject18', 'subject22', 'subject21', 'subject20', 'subject25', 'subject24', 'subject23', 'subject17', 'subject19', 'subject16', 'subject31', 'subject26', 'subject27', 'subject35', 'subject32', 'subject29', 'subject33', 'subject34', 'subject28', 'subject30', 'subject45', 'subject39', 'subject40', 'subject37', 'subject43', 'subject42', 'subject41', 'subject36', 'subject

In [113]:
import pandas as pd
import glob
import os

annotations_dir = '/content/drive/My Drive/fyp dataset egogesture/labels'
videos_dir = '/content/drive/My Drive/fyp dataset egogesture/videos'
desired_labels = [10, 11, 57, 58, 46, 36]

# Minimal directory checks
print(f"Checking directories...")
if not os.path.exists(annotations_dir):
    print(f"Error: Annotations directory {annotations_dir} does not exist.")
if not os.path.exists(videos_dir):
    print(f"Error: Videos directory {videos_dir} does not exist.")

all_annotations = []
for ann_file in glob.glob(os.path.join(annotations_dir, '**', '*.csv'), recursive=True):
    try:
        sample = pd.read_csv(ann_file, nrows=1, sep=',')
        header = 0
        if not sample.iloc[0].astype(str).str.match(r'^\\d+(\\.\\d+)?$').all():
            header = None

        annotations = pd.read_csv(ann_file, sep=',', names=['label', 'start_frame', 'end_frame'], header=header)
        annotations['label'] = pd.to_numeric(annotations['label'], errors='coerce')
        annotations = annotations.dropna(subset=['label'])
        annotations['start_frame'] = pd.to_numeric(annotations['start_frame'], errors='coerce')
        annotations['end_frame'] = pd.to_numeric(annotations['end_frame'], errors='coerce')
        annotations = annotations.dropna(subset=['start_frame', 'end_frame'])

        filtered = annotations[annotations['label'].isin(desired_labels)]
        if not filtered.empty:
            video_base = os.path.basename(ann_file).replace('Group', 'rgb').replace('.csv', '.avi')
            all_annotations.append(filtered.assign(video=video_base))
    except Exception as e:
        print(f"Error loading {ann_file}: {e}")

if all_annotations:
    all_annotations = pd.concat(all_annotations, ignore_index=True)
    print(f"Cell executed successfully. Total combined annotations: {len(all_annotations)}")
else:
    print("Cell executed, but no annotations loaded.")

Checking directories...
Cell executed successfully. Total combined annotations: 1597


In [114]:
import cv2
import numpy as np
import os
import glob
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
extracted_dir = '/content/drive/My Drive/fyp dataset egogesture'
annotations_dir = os.path.join(extracted_dir, 'labels')
videos_dir = os.path.join(extracted_dir, 'videos')
feature_save_dir = extracted_dir

# Load and filter annotations
desired_labels = [10, 11, 57, 58, 46, 36]
all_annotations = []

print(f"Checking annotations directory: {annotations_dir}")
if os.path.exists(annotations_dir):
    print(f"Annotations directory exists. Contents: {os.listdir(annotations_dir)}")
else:
    print(f"Error: Annotations directory {annotations_dir} does not exist.")

print(f"Checking videos directory: {videos_dir}")
if os.path.exists(videos_dir):
    print(f"Videos directory exists. Contents: {os.listdir(videos_dir)}")
else:
    print(f"Error: Videos directory {videos_dir} does not exist.")

for ann_file in glob.glob(os.path.join(annotations_dir, '**', '*.csv'), recursive=True):
    print(f"Processing file: {ann_file}")
    try:
        sample = pd.read_csv(ann_file, nrows=1, sep=',')
        header = 0 if not sample.iloc[0].astype(str).str.match(r'^\d+(\.\d+)?$').all() else None
        annotations = pd.read_csv(ann_file, sep=',', names=['label', 'start_frame', 'end_frame'], header=header)

        annotations['label'] = pd.to_numeric(annotations['label'], errors='coerce')
        annotations = annotations.dropna(subset=['label'])
        annotations['start_frame'] = pd.to_numeric(annotations['start_frame'], errors='coerce')
        annotations['end_frame'] = pd.to_numeric(annotations['end_frame'], errors='coerce')
        annotations = annotations.dropna(subset=['start_frame', 'end_frame'])

        filtered = annotations[annotations['label'].isin(desired_labels)]
        if not filtered.empty:
            print(f"Filtered annotations from {ann_file}:")
            print(filtered.head())
            # Adjust video base to match case and structure
            video_base = os.path.basename(ann_file).replace('Group', 'rgb').replace('.csv', '.avi')
            subject_dir = os.path.basename(os.path.dirname(os.path.dirname(ann_file))).replace('subject', 'Subject')
            scene_dir = os.path.basename(os.path.dirname(ann_file))
            video_path = None
            for subdir in ['videos_1-001', 'videos_2-002', 'videos_3-004', 'videos_4-003', 'videos_5-003']:
                check_path = os.path.join(videos_dir, subdir, subject_dir, scene_dir, 'Color', video_base)
                if os.path.exists(check_path):
                    video_path = check_path
                    break
            if video_path is None:
                print(f"Error: Video file not found for {video_base} in any subdirectory. Skipping.")
                continue
            print(f"Assigning video path: {video_path}")
            all_annotations.append(filtered.assign(video=video_path))
        else:
            print(f"No matching labels found in {ann_file} from desired_labels: {desired_labels}")
    except Exception as e:
        print(f"Error loading or processing {ann_file}: {e}")

if all_annotations:
    all_annotations = pd.concat(all_annotations, ignore_index=True)
    print(f"\nTotal combined annotations: {len(all_annotations)}")
    print("Final combined annotations:")
    print(all_annotations.head())
else:
    all_annotations = pd.DataFrame(columns=['label', 'start_frame', 'end_frame', 'video'])
    print("\nNo annotations were loaded or filtered successfully from any file.")
    print("Final combined annotations is an empty DataFrame.")

def extract_sequences(annotations, videos_dir):
    sequences = []
    for index, row in annotations.iterrows():
        video_path = row['video']
        if not os.path.exists(video_path):
            print(f"Error: Video file not found at {video_path}. Skipping.")
            continue
        print(f"Attempting to open: {video_path}")
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            continue
        start_frame = int(row['start_frame'])
        end_frame = int(row['end_frame'])
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        frames = []
        frame_count = 0
        target_frames = 30

        while frame_count < target_frames and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print(f"Warning: Reached end of video at frame {start_frame + frame_count} in {video_path}")
                break
            if start_frame + frame_count <= end_frame:
                frames.append(cv2.resize(frame, (64, 64)))
                frame_count += 1
            else:
                print(f"Warning: Reached end_frame {end_frame} at frame {start_frame + frame_count} in {video_path}")
                break

        cap.release()

        # Pad or truncate to exactly 30 frames
        if len(frames) < target_frames:
            print(f"Warning: Only {len(frames)} frames extracted from {video_path}. Padding with last frame.")
            last_frame = frames[-1] if frames else np.zeros((64, 64, 3), dtype=np.uint8)
            frames = frames + [last_frame] * (target_frames - len(frames))
        elif len(frames) > target_frames:
            print(f"Warning: Extracted {len(frames)} frames from {video_path}. Truncating to {target_frames}.")
            frames = frames[:target_frames]

        sequences.append(np.array(frames))

    return sequences

# Main execution block
if 'all_annotations' in globals() and not all_annotations.empty:
    print(f"Extracting sequences from {len(all_annotations)} annotations...")
    sequences = extract_sequences(all_annotations, videos_dir)
    print(f"Extracted {len(sequences)} sequences of 30 frames each.")
    # Save the extracted sequences
    os.makedirs(feature_save_dir, exist_ok=True)
    np.save(os.path.join(feature_save_dir, 'all_features.npy'), np.array(sequences))
    print(f"Features saved to: {os.path.join(feature_save_dir, 'all_features.npy')}")
else:
    print("Error: all_annotations is not defined or empty.")

Streaming output truncated to the last 5000 lines.
Assigning video path: /content/drive/My Drive/fyp dataset egogesture/videos/videos_3-004/Subject28/Scene1/Color/rgb2.avi
Processing file: /content/drive/My Drive/fyp dataset egogesture/labels/subject28/Scene1/Group1.csv
Filtered annotations from /content/drive/My Drive/fyp dataset egogesture/labels/subject28/Scene1/Group1.csv:
   label  start_frame  end_frame
9     57          803        843
Assigning video path: /content/drive/My Drive/fyp dataset egogesture/videos/videos_3-004/Subject28/Scene1/Color/rgb1.avi
Processing file: /content/drive/My Drive/fyp dataset egogesture/labels/subject28/Scene1/Group4.csv
Filtered annotations from /content/drive/My Drive/fyp dataset egogesture/labels/subject28/Scene1/Group4.csv:
    label  start_frame  end_frame
0      10          111        158
11     11         1097       1140
Assigning video path: /content/drive/My Drive/fyp dataset egogesture/videos/videos_3-004/Subject28/Scene1/Color/rgb4.avi
Pr

In [115]:
def extract_optical_flow_features(sequence):
    flow_features = []
    for i in range(len(sequence) - 1):
        frame1 = cv2.cvtColor(sequence[i], cv2.COLOR_BGR2GRAY)
        frame2 = cv2.cvtColor(sequence[i + 1], cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        flow_features.append(np.array([np.mean(magnitude), np.mean(angle)]))
    return np.array(flow_features)

if 'sequences' in globals() and sequences:
    print(f"Extracting features from {len(sequences)} sequences...")
    all_features = []
    for i, seq in enumerate(sequences):
        features = extract_optical_flow_features(seq)
        all_features.append(features)
    all_features = np.array(all_features)
    # Save to feature_save_dir
    save_path = os.path.join(feature_save_dir, 'all_features.npy')
    np.save(save_path, all_features)
    print(f"Total features shape: {all_features.shape}")
    print(f"Features saved as {save_path}.")
else:
    print("Error: No sequences extracted.")

Extracting features from 1542 sequences...
Total features shape: (1542, 29, 2)
Features saved as /content/drive/My Drive/fyp dataset egogesture/all_features.npy.


In [116]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import glob
import os
from sklearn.utils import class_weight
from scipy.ndimage.interpolation import shift

# Define the directory where features were saved
feature_save_dir = '/content/drive/My Drive/fyp dataset egogesture'

# Load features from the Drive location
print(f"Attempting to load feature files from: {feature_save_dir}")
feature_files = glob.glob(os.path.join(feature_save_dir, 'all_features.npy'))

print(f"Found {len(feature_files)} feature files.")
if not feature_files:
    print("Error: No feature files found. Please ensure the previous extraction and saving step completed successfully.")
    all_features = []
else:
    all_features = []
    for file in feature_files:
        try:
            batch_data = np.load(file, allow_pickle=True)
            all_features.append(batch_data)
            print(f"Loaded {file} with shape {batch_data.shape}.")
            # Verify frame count consistency
            if batch_data.shape[1] != 30:  # Expecting 30 frames per sequence
                print(f"Warning: Expected 30 frames, got {batch_data.shape[1]} in {file}.")
        except Exception as e:
            print(f"Error loading {file}: {e}")

if all_features:
    try:
        all_features = np.concatenate(all_features) if len(all_features) > 1 else all_features[0]
        print(f"Successfully loaded and concatenated {len(all_features)} feature sequences.")
        print(f"Shape of consolidated features: {all_features.shape}")

        if 'all_annotations' in globals() and all_features.size > 0 and not all_annotations.empty:
            if len(all_features) != len(all_annotations):
                print(f"Warning: Number of extracted feature sequences ({len(all_features)}) does not match the number of annotations ({len(all_annotations)}).")
                min_len = min(len(all_features), len(all_annotations))
                all_features = all_features[:min_len]
                all_annotations = all_annotations.iloc[:min_len]
                print(f"Truncating features and annotations to match length: {min_len}")

            X = all_features
            y = all_annotations['label'].values

            unique_labels = np.unique(y)
            label_map = {label: i for i, label in enumerate(unique_labels)}
            y_mapped = np.array([label_map[label] for label in y])
            y_categorical = to_categorical(y_mapped, num_classes=len(unique_labels))

            # Check class distribution
            class_weights = class_weight.compute_class_weight('balanced', classes=unique_labels, y=y)
            class_weight_dict = dict(enumerate(class_weights))

            print(f"Shape of X before split: {X.shape}")
            print(f"Shape of y_categorical before split: {y_categorical.shape}")
            print(f"Class weights: {class_weight_dict}")

            if X.shape[0] < 2 or y_categorical.shape[0] < 2:
                print("Error: Not enough samples to perform train/test split.")
            else:
                X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_mapped)

                print(f"Shape of X_train: {X_train.shape}")
                print(f"Shape of X_test: {X_test.shape}")
                print(f"Shape of y_train: {y_train.shape}")
                print(f"Shape of y_test: {y_test.shape}")

                expected_input_shape = (X_train.shape[1], X_train.shape[2])
                print(f"LSTM expected input shape: {expected_input_shape}")
                if X_train.shape[1] != 29 and X_train.shape[1] != 30:
                    print(f"Warning: Unexpected feature shape {X_train.shape}. LSTM input shape might be incorrect.")

                # Enhanced model architecture
                model = Sequential([
                    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer='l2'),
                    BatchNormalization(),
                    Dropout(0.3),
                    LSTM(64),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(32, activation='relu', kernel_regularizer='l2'),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(len(unique_labels), activation='softmax')
                ])

                model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

                if X_train.shape[0] < 32:
                    print(f"Warning: Training set size ({X_train.shape[0]}) is less than batch size (32). Adjusting batch_size.")
                    batch_size = max(1, X_train.shape[0])
                else:
                    batch_size = 32

                # Callbacks
                early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
                reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)

                # Optional data augmentation
                # def augment_sequence(seq, shift_amount=1):
                #     return shift(seq, [0, shift_amount, 0], mode='constant', cval=0.0)
                # X_train_aug = np.concatenate([X_train, np.array([augment_sequence(seq) for seq in X_train])])
                # y_train_aug = np.concatenate([y_train, y_train])
                # X_test_aug = np.array([augment_sequence(seq) for seq in X_test])
                # y_test_aug = y_test
                # history = model.fit(X_train_aug, y_train_aug, epochs=50, batch_size=batch_size, validation_data=(X_test_aug, y_test_aug),
                #                   class_weight=class_weight_dict, verbose=1, callbacks=[early_stopping, reduce_lr])
                history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_test, y_test),
                                  class_weight=class_weight_dict, verbose=1, callbacks=[early_stopping, reduce_lr])

                test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
                print(f"Test accuracy: {test_accuracy:.4f}")
                print(f"Test loss: {test_loss:.4f}")
                print(f"Best validation loss: {min(history.history['val_loss'])}")

                os.makedirs(feature_save_dir, exist_ok=True)
                model_save_path = os.path.join(feature_save_dir, 'gesture_recognition_model.h5')
                label_map_save_path = os.path.join(feature_save_dir, 'label_map.npy')
                unique_labels_save_path = os.path.join(feature_save_dir, 'unique_labels.npy')

                model.save(model_save_path)
                np.save(label_map_save_path, label_map)
                np.save(unique_labels_save_path, unique_labels)
                print(f"Model saved to: {model_save_path}")
                print(f"Label map saved to: {label_map_save_path}")
                print(f"Unique labels saved to: {unique_labels_save_path}")
                print("Model and metadata saved.")
        else:
            print("Error: Data not available for training (all_annotations empty or no features loaded).")
    except ValueError as ve:
        print(f"ValueError during concatenation or processing: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred during model training: {e}")
else:
    print("Error: No features loaded from files. Cannot proceed with training.")

Attempting to load feature files from: /content/drive/My Drive/fyp dataset egogesture
Found 1 feature files.
Loaded /content/drive/My Drive/fyp dataset egogesture/all_features.npy with shape (1542, 29, 2).
Successfully loaded and concatenated 1542 feature sequences.
Shape of consolidated features: (1542, 29, 2)
Shape of X before split: (1542, 29, 2)
Shape of y_categorical before split: (1542, 6)
Class weights: {0: np.float64(1.00390625), 1: np.float64(0.9961240310077519), 2: np.float64(1.0), 3: np.float64(1.00390625), 4: np.float64(1.007843137254902), 5: np.float64(0.9884615384615385)}
Shape of X_train: (1233, 29, 2)
Shape of X_test: (309, 29, 2)
Shape of y_train: (1233, 6)
Shape of y_test: (309, 6)
LSTM expected input shape: (29, 2)


<ipython-input-116-3216187579>:11: DeprecationWarning: Please import `shift` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.interpolation import shift
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.1639 - loss: 2.8460 - val_accuracy: 0.1812 - val_loss: 2.2383 - learning_rate: 0.0010
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.1609 - loss: 2.6043 - val_accuracy: 0.1553 - val_loss: 2.2350 - learning_rate: 0.0010
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.1898 - loss: 2.4601 - val_accuracy: 0.1812 - val_loss: 2.2222 - learning_rate: 0.0010
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.2017 - loss: 2.4612 - val_accuracy: 0.1780 - val_loss: 2.2203 - learning_rate: 0.0010
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.1866 - loss: 2.3832 - val_accuracy: 0.1618 - val_loss: 2.2070 - learning_rate: 0.0010
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - accuracy: 0.1906 - loss: 2.3299 - val_accuracy: 0.1683 - val_loss: 2.1831 - learning_rate: 0.0010
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.2018 - loss: 2.3335 - val_

Test accuracy: 0.1748
Test loss: 1.8580
Best validation loss: 1.857964038848877
Model saved to: /content/drive/My Drive/fyp dataset egogesture/gesture_recognition_model.h5
Label map saved to: /content/drive/My Drive/fyp dataset egogesture/label_map.npy
Unique labels saved to: /content/drive/My Drive/fyp dataset egogesture/unique_labels.npy
Model and metadata saved.


In [117]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Confusion Matrix:\n", cm)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
Confusion Matrix:
 [[23  0 11 11  3  3]
 [20  1 11 15  4  1]
 [21  3  7 11  5  5]
 [15  1 14 10  2  9]
 [15  0 10 12  3 11]
 [15  1 12 12  2 10]]


In [118]:
import os
print(os.listdir('/content/'))

['.config', 'drive', 'sample_data']


In [119]:
from google.colab import files
files.download('gesture_recognition_model.h5')
files.download('label_map.npy')
files.download('unique_labels.npy')

FileNotFoundError: Cannot find file: gesture_recognition_model.h5

In [ ]:
import psutil
print(psutil.virtual_memory())